In [4]:
!pip install -q sentence-transformers faiss-cpu gradio python-docx

from docx import Document
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import gradio as gr

# Upload your CV
from google.colab import files
uploaded = files.upload()

# Load and extract text from the CV
doc = Document("Technical Godly CV.docx")
text = "\n".join([para.text for para in doc.paragraphs if para.text.strip()])

# Split text into chunks
def split_text(text, max_length=500):
    sentences = text.split('. ')
    chunks = []
    current_chunk = ""
    for sentence in sentences:
        if len(current_chunk) + len(sentence) < max_length:
            current_chunk += sentence + ". "
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sentence + ". "
    if current_chunk:
        chunks.append(current_chunk.strip())
    return chunks

chunks = split_text(text)

# Load sentence transformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

# Create embeddings
embeddings = model.encode(chunks)

# Create FAISS index
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(np.array(embeddings))

# Define chatbot function
def chatbot(query):
    query_embedding = model.encode([query])
    D, I = index.search(np.array(query_embedding), k=1)
    return chunks[I[0][0]]

# Launch Gradio chatbot
gr.Interface(
    fn=chatbot,
    inputs=gr.Textbox(lines=2, placeholder="Ask about Godly Thomas's experience, skills, or projects..."),
    outputs="text",
    title="Godly Thomas CV Chatbot",
    description="Ask questions about Godly Thomas's CV. Powered by Sentence Transformers + FAISS + Gradio."
).launch()


Saving cleaned_technical_cv.txt to cleaned_technical_cv.txt
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://37d2b7dce06deb6fa1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
